In [89]:
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
from keras.callbacks import LambdaCallback

In [21]:
df = pd.read_csv('PokeDataset.csv')

In [22]:
df = df[['Name', 'Type1', 'Type2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]

In [23]:
df.head()

,Name,Type1,Type2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45
1,Ivysaur,Grass,Poison,60,62,63,80,80,60
2,Venusaur,Grass,Poison,80,82,83,100,100,80
3,Charmander,Fire,NaN,39,52,43,60,50,65
4,Charmeleon,Fire,NaN,58,64,58,80,65,80


In [78]:
names = df['Name'].values
print(names[:5])

['Bulbasaur' 'Ivysaur' 'Venusaur' 'Charmander' 'Charmeleon']


In [79]:
names_duplicates = list(map(lambda s: s.split('-')[0], names))
names = list(set(names_duplicates))
names = list(map(lambda s: s + '.', names))
names = list(map(lambda s: s.lower(), names))
# remove '
names = list(map(lambda s: s.replace("'", ''), names))
# remove numbers
names = list(map(lambda s: ''.join([i for i in s if not i.isdigit()]), names))
# remove :
names = list(map(lambda s: s.replace(":", ''), names))
# change é to e
names = list(map(lambda s: s.replace("é", 'e'), names))
# remove ♂ and ♀
names = list(map(lambda s: s.replace("♂", ''), names))
names = list(map(lambda s: s.replace("♀", ''), names))


In [80]:

char_to_index = dict( (chr(i+96), i) for i in range(1,27))
char_to_index[' '] = 0
char_to_index['.'] = 27

index_to_char = dict( (i, chr(i+96)) for i in range(1,27))
index_to_char[0] = ' '
index_to_char[27] = '.'

max_char = len(max(names, key=len))
m = len(names)
char_dim = len(char_to_index)


In [ ]:
type_dim = 18

In [82]:


X = np.zeros((m, max_char, char_dim))
Y = np.zeros((m, max_char, char_dim))

for i in range(m):
    name = list(names[i])
    for j in range(len(name)):
        X[i, j, char_to_index[name[j]]] = 1
        if j < len(name)-1:
            Y[i, j, char_to_index[name[j+1]]] = 1

In [86]:
X.shape, Y.shape

((904, 13, 28), (904, 13, 28))

In [87]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_char, char_dim), return_sequences=True))
model.add(Dense(char_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [91]:

def make_name(model):
    name = []
    x = np.zeros((1, max_char, char_dim))
    end = False
    i = 0
    
    while end==False:
        probs = list(model.predict(x, verbose=0)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(char_dim), p=probs)
        if i == max_char-2:
            character = '.'
            end = True
        else:
            character = index_to_char[index]
        name.append(character)
        x[0, i+1, index] = 1
        i += 1
        if character == '.':
            end = True
    
    print(''.join(name))

In [92]:
def train(epoch, _):
    if epoch % 25 == 0:
        
        print('Names generated after epoch %d:' % epoch)

        for i in range(3):
            make_name(model)
        
        print()
      
name_generator = LambdaCallback(on_epoch_end = train)

model.fit(X, Y, batch_size=64, epochs=500, callbacks=[name_generator], verbose=0)

Names generated after epoch 0:
elofede.
jolmunenr.
hastoisl.

Names generated after epoch 25:
.
 lavaoeuije.
lelpexo.

Names generated after epoch 50:
arman.
quila.
lechrio.

Names generated after epoch 75:
iglapim.
kolns.
rowooay.

Names generated after epoch 100:
usplam.
.
levamewmo.

Names generated after epoch 125:
cosmiioe.
momqakoo.
ragigqeyodx.

Names generated after epoch 150:
ingenl .
oldagae lpz.
eatkfdono.

Names generated after epoch 175:
laxni.
omton.
eeslur.

Names generated after epoch 200:
 salsjfoh.
elmeopbg.
oreritln.

Names generated after epoch 225:
rimero.
oltitngsgad.
vilicqlzm.

Names generated after epoch 250:
goruruvnoia.
urufasgze.
udeslwt.

Names generated after epoch 275:
faslosz.
ovspemhne.
wisaac.

Names generated after epoch 300:
rawuoksen s.
okpryeaythm.
urtzlo.

Names generated after epoch 325:
macmpzolfee.
wepisldaery.
jniltieyg.

Names generated after epoch 350:
inari.
ukluo.
igli teei.

Names generated after epoch 375:
urblun.
oltonntry.
meowtpx.

Na

In [125]:
make_name(model)

erdurg.
